In [1]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import scipy.stats as stats
import seaborn as sns
import pandas as pd
from causalgraphicalmodels import CausalGraphicalModel
import daft

This is based on the [homework of week 4](https://github.com/rmcelreath/statrethinking_winter2019/blob/master/homework/week04.pdf) of Statistical Rethinking book material.
I do the homework based on the questions but not following a specific order.
<br>
<br>
<br>

In [2]:
islands = {"Bird_A": [0.2, 0.8, 0.05],
           "Bird_B": [0.2, 0.1, 0.15],
           "Bird_C": [0.2, 0.05, 0.7],
           "Bird_D": [0.2, 0.025, 0.05],
           "Bird_E": [0.2, 0.025, 0.05],
          }


df = pd.DataFrame(islands)
df.rename(index={0:'Island_1', 1:'Island_2', 2:'Island_3'}, inplace=True)
df

,Bird_A,Bird_B,Bird_C,Bird_D,Bird_E
Island_1,0.20,0.20,0.20,0.200,0.200
Island_2,0.80,0.10,0.05,0.025,0.025
Island_3,0.05,0.15,0.70,0.050,0.050


## Computing Entropy
Computing the entropy of each island's bird distribution.

In [3]:
def entropy(series):
    return -np.sum((series)*np.log((series)))

Entropy for Island 1:

In [4]:
entropy(df.loc['Island_1'].values)

1.6094379124341005

Entropy for Island 2:

In [5]:
entropy(df.loc['Island_2'].values)

0.7430039367341686

Entropy for Island 3:

In [6]:
entropy(df.loc['Island_3'].values)

0.9836002995230935

The results show that the entropy for Island 1 is greater followed by Island 2 and then Island 3. The higher the entropy the more uniformly the population is distributed.
<br>
<br>
That can be noticed when we compare the bird proportions, in Island 1 it's uniformly distributed. You expect to see any of the birds.
<br>
<br>
In Island 2 there's a 80% population for Bird_A and it would be very surprising to see a bird that's not Bird_A. Furthermore, in Island 3 we have a 70% population for Bird_C. Which makes the entropy of these 2 smaller when compared to Island 1.

## Computing Divergence
Using each island's bird distribution to predict the other two by computing the K-L Divergence.

In [7]:
def divergenceKL(target, model):
    return np.sum(target*np.log(target/model))

Island 1 as the target and Islands 1, 2 and 3 as the model:

In [15]:
dKL_1_1 = divergenceKL(df.loc['Island_1'].values, df.loc['Island_1'].values)
dKL_1_2 = divergenceKL(df.loc['Island_1'].values, df.loc['Island_2'].values)
dKL_1_3 = divergenceKL(df.loc['Island_1'].values, df.loc['Island_3'].values)
print("KL divergence(Island_1, Island_1):", dKL_1_1)
print("KL divergence(Island_1, Island_2):", dKL_1_2)
print("KL divergence(Island_1, Island_3):", dKL_1_3)

KL divergence(Island_1, Island_1): 0.0
KL divergence(Island_1, Island_2): 0.9704060527839233
KL divergence(Island_1, Island_3): 0.638760437463217


Island 2 as the target and Islands 1, 2 and 3 as the model:

In [16]:
dKL_2_1 = divergenceKL(df.loc['Island_2'].values, df.loc['Island_1'].values)
dKL_2_2 = divergenceKL(df.loc['Island_2'].values, df.loc['Island_2'].values)
dKL_2_3 = divergenceKL(df.loc['Island_2'].values, df.loc['Island_3'].values)
print("KL divergence(Island_2, Island_1):", dKL_2_1)
print("KL divergence(Island_2, Island_2):", dKL_2_2)
print("KL divergence(Island_2, Island_3):", dKL_2_3)

KL divergence(Island_2, Island_1): 0.8664339756999315
KL divergence(Island_2, Island_2): 0.0
KL divergence(Island_2, Island_3): 2.010914241472249


Island 3 as the target and Islands 1, 2 and 3 as the model:

In [17]:
dKL_3_1 = divergenceKL(df.loc['Island_3'].values, df.loc['Island_1'].values)
dKL_3_2 = divergenceKL(df.loc['Island_3'].values, df.loc['Island_2'].values)
dKL_3_3 = divergenceKL(df.loc['Island_3'].values, df.loc['Island_3'].values)
print("KL divergence(Island_3, Island_1):", dKL_3_1)
print("KL divergence(Island_3, Island_2):", dKL_3_2)
print("KL divergence(Island_3, Island_3):", dKL_3_3)

KL divergence(Island_3, Island_1): 0.6258376129110066
KL divergence(Island_3, Island_2): 1.8388451788909108
KL divergence(Island_3, Island_3): 0.0


The Island_1 predicts better the others as it has the smallest KL divergence related to the other 2 islands.
<br>
<br>
This is the case because the Island_1 has the highest entropy, their birds species are more uniformly distributed, therefore, it's less surprising to see any species of bird in the other islands as you see all of them in the same proportion in Island_1.
<br>
<br>
Notice that the KL divergence for the same distribution is equal to zero.
<br>
<br>
**Note**: To understand better the property of the KL divergence for why dKL(Island_1, Island_2) is different from dKL(Island_2, Island_1) it's worth reading the short section *Divergence depends upon direction* from the 2nd edition of the book, page 208.

---

---

In [11]:
%load_ext watermark
%watermark -iv -v -nuw

seaborn 0.10.1
daft    0.1.0
numpy   1.19.1
pandas  1.1.1
arviz   0.10.0
pymc3   3.8
last updated: Tue Dec 01 2020 

CPython 3.7.7
IPython 7.17.0
watermark 2.0.2
